# Executar um script de treinamento com SDK do Python

In [ ]:
pip show azure-ai-ml

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
%%writefile src/sorvetes_training.py

import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Argumento de entrada
parser = argparse.ArgumentParser()
parser.add_argument("--data_file", type=str, help="Path to the CSV data file")
args = parser.parse_args()

# Carregamento dos dados
print("Loading Data ...")
sorvetes = pd.read_csv(args.data_file)

# Separar features e labels
X, y = sorvetes[['Temperatura']].values, sorvetes['Vendas'].values

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Hiperparâmetro
reg = 0.01

# Treinamento
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Acurácia
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
print('AUC:', auc)

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
from datetime import datetime


data_input = Input(
    path="./dados/sorvetes_magico.csv",  # caminho local do arquivo
    type="uri_file"
)

job = command(
    name = f"treinamento-sorvetes-{datetime.now().strftime('%Y%m%d%H%M%S')}",
    code="./src",
    command="python sorvetes_training.py --data_file ${{inputs.data_input}}",
    inputs={"data_input": data_input},
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1",
    compute="cpu-geleto-instance",
    experiment_name="sorvetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)